#                     Hospital Management  Schema

## Introduction

The database schema used in this analysis focuses on healthcare-related data and includes multiple interrelated tables that store information about patients, doctors, hospital stays, and appointment details.The purpose of this schema is to manage and analyze key aspects of hospital operations,including patient admissions, appointment scheduling, medical severity levels, and doctor assignments.

The schema consists of the following key tables:
1. patient_data: Stores information about patients, including patient identifiers (key_nis), hospital identifiers (hosp_nis), and demographic and medical data such as age, gender, diagnosis, and total charges.
2. doctor_details: Contains data about doctors, including doctor_id, name, specialty, and contact information.
3. appointment_details: Manages appointments, including the appointment ID, patient (key_nis), hospital (hosp_nis), assigned doctor, appointment date, and purpose.
4. hospital_severity: Tracks the severity of a patient's condition, including the hospital (hosp_nis), patient (key_nis), diagnosis-related group (aprdrg), and severity level (aprdrg_severity).


Purpose of Queries
The purpose of the queries is to retrieve, aggregate, and analyze the data stored across these related tables. Specifically, the queries are designed to:

• Retrieve All Patients with Their Doctors and Severity LevelsAnalyze patient distributions across different severity levels.

• Count the Number of Patients per Severity Level

• Retrieve Patients Admitted to Specific Hospitals

• Find the Total Number of Appointments for Each Doctor

• Retrieve All Critical Severity Patients with Specific Doctor Assignments

• Examine the performance of the queries using EXPLAIN ANALYZE to identify areas for optimization and potential performance improvements through indexing or query design refinements.

These queries provide valuable insights into hospital operations and patient care by leveraging the interconnectedness of the schema.


## 1. Creating Tables

### Hospital table    
CREATE TABLE hospital_info (
    HOSP_BEDSIZE INT,
    HOSP_DIVISION VARCHAR(10),
    HOSP_NIS INT PRIMARY KEY, -- Primary key since HOSP_NIS is unique here
    H_CONTRL VARCHAR(10),
    TOTAL_DISC INT
);

### Patient table
CREATE TABLE patient_data (
    AGE INT,
    DIED INT,
    DRG INT,
    FEMALE VARCHAR(10),
	HOSP_DIVISION VARCHAR(10),
    HOSP_NIS INT, -- Foreign key from the hospital_info table
    I10_DX1 VARCHAR(10), -- ICD-10 diagnosis code
    I10_PR1 VARCHAR(10), -- ICD-10 procedure code
    KEY_NIS INT PRIMARY KEY, -- Primary key for patient data
    LOS INT, -- Length of stay
    PAY1 INT, -- Primary expected payer
    RACE VARCHAR(10), -- Race
    TOTCHG INT, -- Total charges
    CONSTRAINT fk_patient_data_hosp_nis
    FOREIGN KEY (HOSP_NIS) REFERENCES hospital_info (HOSP_NIS) -- Foreign key constraint
);

### Hospital Severity table
CREATE TABLE hospital_severity (
    HOSP_NIS INT,
    KEY_NIS INT, -- Foreign key from the `patient_data` or other table
    APRDRG INT,
    
    APRDRG_Severity VARCHAR(10),
    CONSTRAINT fk_hospital_drg_data_key_nis
    FOREIGN KEY (KEY_NIS) REFERENCES patient_data (KEY_NIS) -- Foreign key constraint
);

### Appointment table
CREATE TABLE public.appointment_details (
    appointment_id SERIAL PRIMARY KEY,
	key_nis INTEGER NOT NULL,
    hosp_nis INTEGER NOT NULL,
    doctor_id INTEGER NOT NULL,
    appointment_date DATE NOT NULL,
    purpose VARCHAR(255),
    FOREIGN KEY (key_nis) REFERENCES public.patient_data(key_nis),
    FOREIGN KEY (doctor_id) REFERENCES public.doctor_details(doctor_id)
);

### Doctor table
CREATE TABLE public.doctor_details (
    doctor_id SERIAL PRIMARY KEY,
    doctor_name VARCHAR(100) NOT NULL,
    specialty VARCHAR(100) NOT NULL,
    phone_number VARCHAR(15),
    email VARCHAR(100),
    hosp_nis INTEGER,
    FOREIGN KEY (hosp_nis) REFERENCES public.hospital_info(hosp_nis)
);

## Inserting Rows

### Hospital Table
INSERT INTO public.hospital_info (hosp_nis, hosp_bedsize, hosp_division, h_contrl, total_disc)
VALUES 
(1, 500, 'East Division', 'Private', 200),
(2, 250, 'West Division', 'Government', 150),
(3, 350, 'South Division', 'Private', 180),
(4, 400, 'North Division', 'Government', 220),
(5, 300, 'Central Division', 'Private', 190),
(6, 450, 'West Division', 'Private', 250),
(7, 200, 'East Division', 'Government', 100),
(8, 600, 'South Division', 'Private', 300),
(9, 550, 'North Division', 'Government', 280),
(10, 500, 'Central Division', 'Private', 210);

### Patient data
INSERT INTO public.patient_data (hosp_nis, age, died, drg, female, hosp_division, i10_dx1, i10_pr1, key_nis, los, pay1, race, totchg) 
VALUES 
(1, 68, 0, 101, 'F', 'East Division', 'A01', 'B01', 1001, 5, 1, 1, 20000),
(2, 54, 0, 102, 'M', 'West Division', 'A02', 'B02', 1002, 3, 2, 2, 15000),
(3, 45, 0, 103, 'F', 'South Division', 'A03', 'B03', 1003, 7, 1, 3, 18000),
(4, 29, 1, 104, 'M', 'North Division', 'A04', 'B04', 1004, 6, 3, 2, 22000),
(5, 39, 0, 105, 'F', 'Central Division', 'A05', 'B05', 1005, 4, 2, 1, 13000),
(6, 50, 0, 106, 'M', 'West Division', 'A06', 'B06', 1006, 8, 3, 3, 17500),
(7, 72, 0, 107, 'F', 'East Division', 'A07', 'B07', 1007, 5, 2, 2, 19000),
(8, 60, 0, 108, 'M', 'South Division', 'A08', 'B08', 1008, 7, 1, 1, 22000),
(9, 33, 0, 109, 'F', 'North Division', 'A09', 'B09', 1009, 3, 3, 3, 16000),
(10, 48, 1, 110, 'M', 'Central Division', 'A10', 'B10', 1010, 4, 2, 2, 20000),
(1, 51, 0, 111, 'M', 'East Division', 'A11', 'B11', 1011, 6, 1, 3, 23000),
(2, 59, 0, 112, 'F', 'West Division', 'A12', 'B12', 1012, 3, 2, 1, 18000),
(3, 42, 0, 113, 'M', 'South Division', 'A13', 'B13', 1013, 5, 1, 2, 17000),
(4, 30, 0, 114, 'F', 'North Division', 'A14', 'B14', 1014, 4, 3, 3, 15000),
(5, 66, 1, 115, 'M', 'Central Division', 'A15', 'B15', 1015, 7, 2, 1, 20000),
(6, 44, 0, 116, 'F', 'West Division', 'A16', 'B16', 1016, 3, 1, 3, 21000),
(7, 38, 0, 117, 'M', 'East Division', 'A17', 'B17', 1017, 6, 2, 2, 25000),
(8, 55, 0, 118, 'F', 'South Division', 'A18', 'B18', 1018, 5, 3, 1, 14000),
(9, 63, 0, 119, 'M', 'North Division', 'A19', 'B19', 1019, 3, 1, 2, 19000),
(10, 36, 1, 120, 'F', 'Central Division', 'A20', 'B20', 1020, 4, 3, 3, 12000),
(1, 28, 0, 121, 'F', 'East Division', 'A21', 'B21', 1021, 7, 2, 1, 21000),
(2, 69, 0, 122, 'M', 'West Division', 'A22', 'B22', 1022, 6, 3, 3, 20000),
(3, 40, 1, 123, 'F', 'South Division', 'A23', 'B23', 1023, 3, 1, 2, 17000),
(4, 53, 0, 124, 'M', 'North Division', 'A24', 'B24', 1024, 5, 2, 1, 14000),
(5, 35, 0, 125, 'F', 'Central Division', 'A25', 'B25', 1025, 4, 3, 3, 16000),
(6, 67, 1, 126, 'M', 'West Division', 'A26', 'B26', 1026, 6, 1, 2, 15000),
(7, 61, 0, 127, 'F', 'East Division', 'A27', 'B27', 1027, 5, 2, 1, 18000),
(8, 29, 1, 128, 'M', 'South Division', 'A28', 'B28', 1028, 4, 3, 3, 19000),
(9, 47, 0, 129, 'F', 'North Division', 'A29', 'B29', 1029, 7, 2, 2, 20000),
(10, 41, 0, 130, 'M', 'Central Division', 'A30', 'B30', 1030, 3, 1, 3, 17000),
(1, 39, 1, 131, 'F', 'East Division', 'A31', 'B31', 1031, 5, 2, 1, 16000),
(2, 62, 0, 132, 'M', 'West Division', 'A32', 'B32', 1032, 4, 3, 3, 15000),
(3, 58, 1, 133, 'F', 'South Division', 'A33', 'B33', 1033, 6, 1, 2, 22000),
(4, 32, 0, 134, 'M', 'North Division', 'A34', 'B34', 1034, 7, 2, 1, 23000),
(5, 45, 0, 135, 'F', 'Central Division', 'A35', 'B35', 1035, 3, 3, 3, 14000),
(6, 34, 0, 136, 'M', 'West Division', 'A36', 'B36', 1036, 5, 2, 1, 15000),
(7, 48, 1, 137, 'F', 'East Division', 'A37', 'B37', 1037, 6, 1, 2, 12000),
(8, 30, 0, 138, 'M', 'South Division', 'A38', 'B38', 1038, 4, 3, 3, 14000),
(9, 70, 0, 139, 'F', 'North Division', 'A39', 'B39', 1039, 7, 2, 1, 18000),
(10, 33, 0, 140, 'M', 'Central Division', 'A40', 'B40', 1040, 5, 1, 2, 16000),
(1, 56, 0, 141, 'F', 'East Division', 'A41', 'B41', 1041, 3, 3, 3, 19000),
(2, 25, 0, 142, 'M', 'West Division', 'A42', 'B42', 1042, 4, 2, 2, 17000),
(3, 51, 0, 143, 'F', 'South Division', 'A43', 'B43', 1043, 6, 1, 1, 16000),
(4, 64, 1, 144, 'M', 'North Division', 'A44', 'B44', 1044, 7, 2, 3, 21000),
(5, 49, 0, 145, 'F', 'Central Division', 'A45', 'B45', 1045, 4, 2, 1, 23000),
(6, 59, 1, 146, 'M', 'West Division', 'A46', 'B46', 1046, 5, 1, 2, 19000),
(7, 34, 0, 147, 'F', 'East Division', 'A47', 'B47', 1047, 6, 3, 3, 15000),
(8, 50, 0, 148, 'M', 'South Division', 'A48', 'B48', 1048, 3, 2, 2, 16000),
(9, 40, 0, 149, 'F', 'North Division', 'A49', 'B49', 1049, 7, 1, 1, 22000),
(10, 67, 1, 150, 'M', 'Central Division', 'A50', 'B50', 1050, 5, 3, 3, 24000);

### Hospital_severity
INSERT INTO public.hospital_severity (hosp_nis, key_nis, aprdrg, aprdrg_severity) 
VALUES
(1, 1001, 101, 'Moderate'),
(2, 1002, 102, 'Severe'),
(3, 1003, 103, 'Low'),
(4, 1004, 104, 'Critical'),
(5, 1005, 105, 'Moderate'),
(6, 1006, 106, 'Severe'),
(7, 1007, 107, 'Low'),
(8, 1008, 108, 'Critical'),
(9, 1009, 109, 'Moderate'),
(10, 1010, 110, 'Severe'),
(1, 1011, 111, 'Low'),
(2, 1012, 112, 'Moderate'),
(3, 1013, 113, 'Critical'),
(4, 1014, 114, 'Low'),
(5, 1015, 115, 'Moderate'),
(6, 1016, 116, 'Severe'),
(7, 1017, 117, 'Critical'),
(8, 1018, 118, 'Low'),
(9, 1019, 119, 'Moderate'),
(10, 1020, 120, 'Severe'),
(1, 1021, 121, 'Critical'),
(2, 1022, 122, 'Low'),
(3, 1023, 123, 'Moderate'),
(4, 1024, 124, 'Severe'),
(5, 1025, 125, 'Critical'),
(6, 1026, 126, 'Low'),
(7, 1027, 127, 'Moderate'),
(8, 1028, 128, 'Severe'),
(9, 1029, 129, 'Critical'),
(10, 1030, 130, 'Low'),
(1, 1031, 131, 'Moderate'),
(2, 1032, 132, 'Severe'),
(3, 1033, 133, 'Critical'),
(4, 1034, 134, 'Low'),
(5, 1035, 135, 'Moderate'),
(6, 1036, 136, 'Severe'),
(7, 1037, 137, 'Critical'),
(8, 1038, 138, 'Low'),
(9, 1039, 139, 'Moderate'),
(10, 1040, 140, 'Severe'),
(1, 1041, 141, 'Critical'),
(2, 1042, 142, 'Low'),
(3, 1043, 143, 'Moderate'),
(4, 1044, 144, 'Severe'),
(5, 1045, 145, 'Critical'),
(6, 1046, 146, 'Low'),
(7, 1047, 147, 'Moderate'),
(8, 1048, 148, 'Severe'),
(9, 1049, 149, 'Critical'),
(10, 1050, 150, 'Moderate');

### Doctor details

INSERT INTO public.doctor_details (doctor_name, specialty, phone_number, email, hosp_nis)
VALUES 
('Dr. John Smith', 'Cardiology', '555-1234', 'john.smith@hospital.com', 1),
('Dr. Emily Brown', 'Neurology', '555-5678', 'emily.brown@hospital.com', 2),
('Dr. Michael Davis', 'Orthopedics', '555-9876', 'michael.davis@hospital.com', 3),
('Dr. Laura White', 'Pediatrics', '555-5432', 'laura.white@hospital.com', 4),
('Dr. David Harris', 'General Surgery', '555-6543', 'david.harris@hospital.com', 5),
('Dr. Alice Green', 'Dermatology', '555-7654', 'alice.green@hospital.com', 6),
('Dr. Jack Wilson', 'ENT', '555-8765', 'jack.wilson@hospital.com', 7),
('Dr. Sarah Johnson', 'Radiology', '555-9870', 'sarah.johnson@hospital.com', 8),
('Dr. Robert Lee', 'Anesthesiology', '555-0987', 'robert.lee@hospital.com', 9),
('Dr. Jennifer Taylor', 'Cardiology', '555-5674', 'jennifer.taylor@hospital.com', 10),
('Dr. James Brown', 'Neurology', '555-5688', 'james.brown@hospital.com', 1),
('Dr. Karen Black', 'Orthopedics', '555-5567', 'karen.black@hospital.com', 2),
('Dr. Steve Young', 'Pediatrics', '555-5654', 'steve.young@hospital.com', 3),
('Dr. Susan Green', 'General Surgery', '555-6540', 'susan.green@hospital.com', 4),
('Dr. Mark King', 'Dermatology', '555-9001', 'mark.king@hospital.com', 5),
('Dr. Emma Collins', 'ENT', '555-8541', 'emma.collins@hospital.com', 6),
('Dr. Brian White', 'Radiology', '555-7563', 'brian.white@hospital.com', 7),
('Dr. Olivia Moore', 'Anesthesiology', '555-1222', 'olivia.moore@hospital.com', 8),
('Dr. Daniel Evans', 'Orthopedics', '555-1333', 'daniel.evans@hospital.com', 9),
('Dr. Grace Allen', 'Pediatrics', '555-1444', 'grace.allen@hospital.com', 10);

### Appointment_details

INSERT INTO public.appointment_details (key_nis, hosp_nis, doctor_id, appointment_date, purpose) 
VALUES
(1001, 1, 1, '2024-09-20', 'Follow-up for heart condition'),
(1002, 2, 2, '2024-09-21', 'Consultation for migraine'),
(1003, 3, 3, '2024-09-22', 'Back pain treatment'),
(1004, 4, 4, '2024-09-23', 'Routine check-up'),
(1005, 5, 5, '2024-09-24', 'Surgery follow-up'),
(1006, 6, 6, '2024-09-25', 'Skin condition consultation'),
(1007, 7, 7, '2024-09-26', 'ENT consultation'),
(1008, 8, 8, '2024-09-27', 'Radiology scan'),
(1009, 9, 9, '2024-09-28', 'Anesthesia pre-surgery consult'),
(1010, 10, 10, '2024-09-29', 'Cardiology consultation'),
(1011, 1, 11, '2024-09-30', 'Follow-up for heart condition'),
(1012, 2, 12, '2024-10-01', 'Consultation for migraine'),
(1013, 3, 13, '2024-10-02', 'Back pain treatment'),
(1014, 4, 14, '2024-10-03', 'Routine check-up'),
(1015, 5, 15, '2024-10-04', 'Surgery follow-up'),
(1016, 6, 16, '2024-10-05', 'Skin condition consultation'),
(1017, 7, 17, '2024-10-06', 'ENT consultation'),
(1018, 8, 18, '2024-10-07', 'Radiology scan'),
(1019, 9, 19, '2024-10-08', 'Anesthesia pre-surgery consult'),
(1020, 10, 20, '2024-10-09', 'Cardiology consultation'),
(1021, 1, 1, '2024-10-10', 'Consultation for heart condition'),
(1022, 2, 2, '2024-10-11', 'Consultation for migraine'),
(1023, 3, 3, '2024-10-12', 'Back pain follow-up'),
(1024, 4, 4, '2024-10-13', 'Routine child check-up'),
(1025, 5, 5, '2024-10-14', 'Post-surgery consultation'),
(1026, 6, 6, '2024-10-15', 'Follow-up for skin condition'),
(1027, 7, 7, '2024-10-16', 'ENT surgery follow-up'),
(1028, 8, 8, '2024-10-17', 'Routine radiology check'),
(1029, 9, 9, '2024-10-18', 'Pre-surgery anesthesia consultation'),
(1030, 10, 10, '2024-10-19', 'Cardiology follow-up'),
(1031, 1, 11, '2024-10-20', 'Heart condition consultation'),
(1032, 2, 12, '2024-10-21', 'Migraine treatment follow-up'),
(1033, 3, 13, '2024-10-22', 'Back pain consultation'),
(1034, 4, 14, '2024-10-23', 'Routine check-up for child'),
(1035, 5, 15, '2024-10-24', 'Post-surgery treatment'),
(1036, 6, 16, '2024-10-25', 'Skin condition follow-up'),
(1037, 7, 17, '2024-10-26', 'ENT follow-up'),
(1038, 8, 18, '2024-10-27', 'Radiology scan follow-up'),
(1039, 9, 19, '2024-10-28', 'Anesthesia pre-op consultation'),
(1040, 10, 20, '2024-10-29', 'Cardiology surgery follow-up'),
(1041, 1, 1, '2024-10-30', 'Heart treatment follow-up'),
(1042, 2, 2, '2024-10-31', 'Consultation for migraine follow-up'),
(1043, 3, 3, '2024-11-01', 'Back pain consultation'),
(1044, 4, 4, '2024-11-02', 'Child routine check-up'),
(1045, 5, 5, '2024-11-03', 'Post-surgery consultation'),
(1046, 6, 6, '2024-11-04', 'Skin condition treatment'),
(1047, 7, 7, '2024-11-05', 'ENT treatment consultation'),
(1048, 8, 8, '2024-11-06', 'Radiology follow-up'),
(1049, 9, 9, '2024-11-07', 'Pre-surgery anesthesia consultation'),
(1050, 10, 10, '2024-11-08', 'Cardiology follow-up');

### Query 1: Retrieve All Patients with Their Doctors and Severity Levels

EXPLAIN ANALYZE 
SELECT p.hosp_nis, p.key_nis, d.doctor_name, hs.aprdrg_severity
FROM patient_data p
JOIN appointment_details ad ON p.key_nis = ad.key_nis
JOIN doctor_details d ON ad.doctor_id = d.doctor_id
JOIN hospital_severity hs ON p.key_nis = hs.key_nis;

Result

#### QUERY PLAN

Hash Join  (cost=19.93..33.39 rows=140 width=230) (actual time=0.855..0.921 rows=50 loops=1)
  Hash Cond: (ad.key_nis = p.key_nis)
  ->  Hash Join  (cost=15.70..28.89 rows=99 width=230) (actual time=0.114..0.166 rows=50 loops=1)
        Hash Cond: (ad.doctor_id = d.doctor_id)
        ->  Hash Join  (cost=3.23..16.14 rows=99 width=16) (actual time=0.064..0.102 rows=50 loops=1)
              Hash Cond: (ad.key_nis = hs.key_nis)
              ->  Seq Scan on appointment_details ad  (cost=0.00..11.40 rows=140 width=8) (actual time=0.013..0.017 rows=50 loops=1)
              ->  Hash  (cost=1.99..1.99 rows=99 width=8) (actual time=0.040..0.040 rows=99 loops=1)
                    Buckets: 1024  Batches: 1  Memory Usage: 13kB
                    ->  Seq Scan on hospital_severity hs  (cost=0.00..1.99 rows=99 width=8) (actual time=0.008..0.020 rows=99 loops=1)
        ->  Hash  (cost=11.10..11.10 rows=110 width=222) (actual time=0.014..0.014 rows=20 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 10kB
              ->  Seq Scan on doctor_details d  (cost=0.00..11.10 rows=110 width=222) (actual time=0.005..0.008 rows=20 loops=1)
  ->  Hash  (cost=2.99..2.99 rows=99 width=8) (actual time=0.323..0.324 rows=99 loops=1)
        Buckets: 1024  Batches: 1  Memory Usage: 12kB
        ->  Seq Scan on patient_data p  (cost=0.00..2.99 rows=99 width=8) (actual time=0.014..0.109 rows=99 loops=1)
        
Planning Time: 3.572 ms
Execution Time: 0.999 ms

#### ANALYSIS

1.	Hash Joins are used for joining multiple tables (appointment_details, hospital_severity, doctor_details, and patient_data). These joins are efficient because hash joins are typically used when both tables are fully scanned, and the data is relatively small.
2.	Sequential Scans are performed on all the tables. Since the dataset is small, the query planner determined that sequential scans are faster than using indexes in this case.
3.	Execution time is 0.999ms


#### With Indexing

CREATE INDEX idx_patient_data_key_nis ON patient_data (key_nis);
CREATE INDEX idx_appointment_details_key_nis ON appointment_details (key_nis);
CREATE INDEX idx_appointment_details_doctor_id ON appointment_details (doctor_id);
CREATE INDEX idx_doctor_details_doctor_id ON doctor_details (doctor_id);
CREATE INDEX idx_hospital_severity_key_nis ON hospital_severity (key_nis);

EXPLAIN ANALYZE
SELECT p.hosp_nis, p.key_nis, d.doctor_name, hs.aprdrg_severity
FROM patient_data p
JOIN appointment_details ad ON p.key_nis = ad.key_nis
JOIN doctor_details d ON ad.doctor_id = d.doctor_id
JOIN hospital_severity hs ON p.key_nis = hs.key_nis;


Result

#### QUERY PLAN

Hash Join  (cost=7.80..10.95 rows=50 width=230) (actual time=0.151..0.205 rows=50 loops=1)
  Hash Cond: (ad.doctor_id = d.doctor_id)
  ->  Hash Join  (cost=6.35..9.36 rows=50 width=16) (actual time=0.118..0.157 rows=50 loops=1)
        Hash Cond: (ad.key_nis = p.key_nis)
        ->  Hash Join  (cost=2.12..4.99 rows=50 width=16) (actual time=0.050..0.073 rows=50 loops=1)
              Hash Cond: (hs.key_nis = ad.key_nis)
              ->  Seq Scan on hospital_severity hs  (cost=0.00..1.99 rows=99 width=8) (actual time=0.002..0.011 rows=99 loops=1)
              ->  Hash  (cost=1.50..1.50 rows=50 width=8) (actual time=0.022..0.023 rows=50 loops=1)
                    Buckets: 1024  Batches: 1  Memory Usage: 10kB
                    ->  Seq Scan on appointment_details ad  (cost=0.00..1.50 rows=50 width=8) (actual time=0.003..0.011 rows=50 loops=1)
        ->  Hash  (cost=2.99..2.99 rows=99 width=8) (actual time=0.048..0.048 rows=99 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 12kB
              ->  Seq Scan on patient_data p  (cost=0.00..2.99 rows=99 width=8) (actual time=0.004..0.022 rows=99 loops=1)
  ->  Hash  (cost=1.20..1.20 rows=20 width=222) (actual time=0.021..0.021 rows=20 loops=1)
        Buckets: 1024  Batches: 1  Memory Usage: 10kB
        ->  Seq Scan on doctor_details d  (cost=0.00..1.20 rows=20 width=222) (actual time=0.008..0.011 rows=20 loops=1)
        
Planning Time: 2.780 ms
Execution Time: 0.253 ms


#### ANALYSIS

1.	Efficient Hash Joins: The hash joins used in this query are highly efficient given the data sizes. The costs are relatively low, and execution times are fast, indicating that the joins are optimal.
2.	Sequential Scans: The plan shows sequential scans on all tables, which is appropriate when the data size is small or when indexes are not available. The sequential scans are fast and do not seem to cause performance bottlenecks in this query.
3.	Execution Time: The total execution time is 0.253 ms, which is very fast. This indicates that the query is efficient, likely due to the relatively small size of the tables involved.

### Query 2: Count the Number of Patients per Severity Level

EXPLAIN ANALYZE 
SELECT hs.aprdrg_severity, COUNT(p.key_nis) AS patient_count
FROM patient_data p
JOIN hospital_severity hs ON p.key_nis = hs.key_nis
GROUP BY hs.aprdrg_severity;

Result

##### QUERY PLAN

HashAggregate  (cost=6.98..7.06 rows=8 width=12) (actual time=0.742..0.747 rows=8 loops=1)
  Group Key: hs.aprdrg_severity
  Batches: 1  Memory Usage: 24kB
  ->  Hash Join  (cost=4.23..6.49 rows=99 width=8) (actual time=0.044..0.104 rows=99 loops=1)
        Hash Cond: (hs.key_nis = p.key_nis)
        ->  Seq Scan on hospital_severity hs  (cost=0.00..1.99 rows=99 width=8) (actual time=0.006..0.021 rows=99 loops=1)
        ->  Hash  (cost=2.99..2.99 rows=99 width=4) (actual time=0.021..0.022 rows=99 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 12kB
              ->  Seq Scan on patient_data p  (cost=0.00..2.99 rows=99 width=4) (actual time=0.004..0.012 rows=99 loops=1)
              
Planning Time: 0.279 ms
Execution Time: 0.830 ms


##### ANALYSIS

1. HashAggregate: This node is responsible for the GROUP BY operation. It efficiently aggregates the data by the aprdrg_severity field and counts the number of rows per severity level.
2. Hash Join: The use of a Hash Join indicates that both tables (hospital_severity and patient_data) are fully scanned and then joined using an in-memory hash table. This is efficient when both tables are relatively small and can fit into memory.
3. Sequential Scans: Both the hospital_severity and patient_data tables were scanned sequentially. This is expected for small datasets, as using an index could be less efficient when the entire table can be read quickly.

#### WITH INDEXING

CREATE INDEX idx_patient_data_key_nis ON patient_data (key_nis);
CREATE INDEX idx_hospital_severity_key_nis ON hospital_severity (key_nis);
CREATE INDEX idx_hospital_severity_aprdrg_severity ON hospital_severity (aprdrg_severity);
EXPLAIN ANALYZE
SELECT hs.aprdrg_severity, COUNT(p.key_nis) AS patient_count
FROM patient_data p
JOIN hospital_severity hs ON p.key_nis = hs.key_nis
GROUP BY hs.aprdrg_severity;

RESULT

#### QUERY PLAN

HashAggregate  (cost=6.98..7.06 rows=8 width=12) (actual time=0.266..0.271 rows=8 loops=1)
  Group Key: hs.aprdrg_severity
  Batches: 1  Memory Usage: 24kB
  ->  Hash Join  (cost=4.23..6.49 rows=99 width=8) (actual time=0.054..0.093 rows=99 loops=1)
        Hash Cond: (hs.key_nis = p.key_nis)
        ->  Seq Scan on hospital_severity hs  (cost=0.00..1.99 rows=99 width=8) (actual time=0.006..0.014 rows=99 loops=1)
        ->  Hash  (cost=2.99..2.99 rows=99 width=4) (actual time=0.037..0.038 rows=99 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 12kB
              ->  Seq Scan on patient_data p  (cost=0.00..2.99 rows=99 width=4) (actual time=0.005..0.020 rows=99 loops=1)
              
Planning Time: 0.437 ms
Execution Time: 0.313 ms



#### ANALYSIS

•  Efficient Aggregation: The hash-based aggregation (HashAggregate) for grouping the data by aprdrg_severity is fast and consumes minimal resources.
•  Efficient Joins: The query uses a hash join to efficiently combine the hospital_severity and patient_data tables. The join is performed efficiently, with both tables being scanned sequentially.
•  Sequential Scans: The sequential scans on both hospital_severity and patient_data are fast due to the small dataset size (99 rows). For larger datasets, an index on the key_nis column would improve performance by avoiding full table scans.
Execution time reduced from 0.830 to 0.313


### Query 3: Retrieve Patients Admitted to Specific Hospitals

EXPLAIN ANALYZE 
SELECT p.hosp_nis, p.key_nis, d.doctor_name, hs.aprdrg_severity
FROM patient_data p
JOIN appointment_details ad ON p.key_nis = ad.key_nis
JOIN doctor_details d ON ad.doctor_id = d.doctor_id
JOIN hospital_severity hs ON p.key_nis = hs.key_nis
WHERE p.hosp_nis > 5;


Result

#### QUERY PLAN

Hash Join  (cost=19.82..32.78 rows=105 width=230) (actual time=0.212..0.246 rows=25 loops=1)
  Hash Cond: (ad.doctor_id = d.doctor_id)
  ->  Hash Join  (cost=7.35..20.01 rows=105 width=16) (actual time=0.187..0.211 rows=25 loops=1)
        Hash Cond: (ad.key_nis = p.key_nis)
        ->  Seq Scan on appointment_details ad  (cost=0.00..11.40 rows=140 width=8) (actual time=0.019..0.026 rows=50 loops=1)
        ->  Hash  (cost=6.42..6.42 rows=74 width=16) (actual time=0.154..0.156 rows=74 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 12kB
              ->  Hash Join  (cost=4.16..6.42 rows=74 width=16) (actual time=0.081..0.130 rows=74 loops=1)
                    Hash Cond: (hs.key_nis = p.key_nis)
                    ->  Seq Scan on hospital_severity hs  (cost=0.00..1.99 rows=99 width=8) (actual time=0.008..0.020 rows=99 loops=1)
                    ->  Hash  (cost=3.24..3.24 rows=74 width=8) (actual time=0.065..0.066 rows=74 loops=1)
                          Buckets: 1024  Batches: 1  Memory Usage: 11kB
                          ->  Seq Scan on patient_data p  (cost=0.00..3.24 rows=74 width=8) (actual time=0.012..0.040 rows=74 loops=1)
                                Filter: (hosp_nis > 5)
                                Rows Removed by Filter: 25
  ->  Hash  (cost=11.10..11.10 rows=110 width=222) (actual time=0.018..0.019 rows=20 loops=1)
        Buckets: 1024  Batches: 1  Memory Usage: 10kB
        ->  Seq Scan on doctor_details d  (cost=0.00..11.10 rows=110 width=222) (actual time=0.007..0.011 rows=20 loops=1)
        
Planning Time: 0.867 ms
Execution Time: 0.381 ms

#### ANALYSIS

1. Hash Join: Multiple hash joins are used to join the tables (patient_data, hospital_severity, appointment_details, doctor_details), which is efficient for smaller datasets where all data can fit into memory.
2.	Sequential Scans: Sequential scans are used for all the tables. This is optimal for small datasets because a sequential scan can read the entire table quickly without the overhead of indexing.


#### AFTER INDEXING

CREATE INDEX idx_patient_data_key_nis ON patient_data (key_nis);
CREATE INDEX idx_patient_data_hosp_nis ON patient_data (hosp_nis);
CREATE INDEX idx_appointment_details_key_nis ON appointment_details (key_nis);
CREATE INDEX idx_appointment_details_doctor_id ON appointment_details (doctor_id);
CREATE INDEX idx_doctor_details_doctor_id ON doctor_details (doctor_id);
CREATE INDEX idx_hospital_severity_key_nis ON hospital_severity (key_nis);
EXPLAIN ANALYZE
SELECT p.hosp_nis, p.key_nis, d.doctor_name, hs.aprdrg_severity
FROM patient_data p
JOIN appointment_details ad ON p.key_nis = ad.key_nis
JOIN doctor_details d ON ad.doctor_id = d.doctor_id
JOIN hospital_severity hs ON p.key_nis = hs.key_nis
WHERE p.hosp_nis > 5;

RESULT

#### QUERY PLAN

Hash Join  (cost=7.72..10.56 rows=37 width=230) (actual time=0.122..0.147 rows=25 loops=1)
  Hash Cond: (ad.doctor_id = d.doctor_id)
  ->  Hash Join  (cost=6.27..9.00 rows=37 width=16) (actual time=0.095..0.112 rows=25 loops=1)
        Hash Cond: (hs.key_nis = p.key_nis)
        ->  Seq Scan on hospital_severity hs  (cost=0.00..1.99 rows=99 width=8) (actual time=0.002..0.011 rows=99 loops=1)
        ->  Hash  (cost=5.80..5.80 rows=37 width=16) (actual time=0.078..0.079 rows=25 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 10kB
              ->  Hash Join  (cost=4.16..5.80 rows=37 width=16) (actual time=0.055..0.070 rows=25 loops=1)
                    Hash Cond: (ad.key_nis = p.key_nis)
                    ->  Seq Scan on appointment_details ad  (cost=0.00..1.50 rows=50 width=8) (actual time=0.004..0.008 rows=50 loops=1)
                    ->  Hash  (cost=3.24..3.24 rows=74 width=8) (actual time=0.042..0.042 rows=74 loops=1)
                          Buckets: 1024  Batches: 1  Memory Usage: 11kB
                          ->  Seq Scan on patient_data p  (cost=0.00..3.24 rows=74 width=8) (actual time=0.005..0.025 rows=74 loops=1)
                                Filter: (hosp_nis > 5)
                                Rows Removed by Filter: 25
  ->  Hash  (cost=1.20..1.20 rows=20 width=222) (actual time=0.021..0.022 rows=20 loops=1)
        Buckets: 1024  Batches: 1  Memory Usage: 10kB
        ->  Seq Scan on doctor_details d  (cost=0.00..1.20 rows=20 width=222) (actual time=0.009..0.012 rows=20 loops=1)
        
Planning Time: 0.742 ms
Execution Time: 0.182 ms


#### ANALYSIS

•  Efficient Hash Joins: The query relies on hash joins for combining the tables, and these joins are very efficient, especially given the relatively small data sizes.
•  Sequential Scans: Sequential scans are used on all the tables, but this is not a performance issue in this case since all the tables have small row counts (99, 50, 74, and 20 rows respectively). For larger datasets, these sequential scans could become a bottleneck, and indexing might be required.
•  Filtering: The WHERE clause (hosp_nis > 5) filters out 25 rows from patient_data, allowing the query to process fewer rows. This operation is handled efficiently with sequential scanning.

Execution time reduced from 0.381 to 0.182

### Query 4: Find the Total Number of Appointments for Each Doctor

EXPLAIN ANALYZE 
SELECT d.doctor_name, COUNT(ad.appointment_id) AS total_appointments
FROM doctor_details d
JOIN appointment_details ad ON d.doctor_id = ad.doctor_id
GROUP BY d.doctor_name;

Result

#### QUERY PLAN

HashAggregate  (cost=24.95..26.05 rows=110 width=226) (actual time=0.369..0.380 rows=20 loops=1)
  Group Key: d.doctor_name
  Batches: 1  Memory Usage: 24kB
  ->  Hash Join  (cost=12.47..24.25 rows=140 width=222) (actual time=0.221..0.258 rows=50 loops=1)
        Hash Cond: (ad.doctor_id = d.doctor_id)
        ->  Seq Scan on appointment_details ad  (cost=0.00..11.40 rows=140 width=8) (actual time=0.030..0.038 rows=50 loops=1)
        ->  Hash  (cost=11.10..11.10 rows=110 width=222) (actual time=0.118..0.119 rows=20 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 10kB
              ->  Seq Scan on doctor_details d  (cost=0.00..11.10 rows=110 width=222) (actual time=0.090..0.097 rows=20 loops=1)
              
Planning Time: 0.363 ms
Execution Time: 0.688 ms

#### ANALYSIS

1. HashAggregate: This operation groups the data by doctor_name and calculates the total number of appointments for each doctor. Since the dataset is small, the aggregation is fast and efficient.
2. Hash Join: The join between appointment_details and doctor_details is handled by a Hash Join, which is efficient for small datasets like this.
3. Sequential Scans: Sequential scans are used for both appointment_details and doctor_details. This is optimal for small datasets because sequential scans are generally faster than using indexes for small tables.

#### AFTER INDEXING

CREATE INDEX idx_doctor_details_doctor_id ON doctor_details (doctor_id);
CREATE INDEX idx_appointment_details_doctor_id ON appointment_details (doctor_id);
EXPLAIN ANALYZE
SELECT d.doctor_name, COUNT(ad.appointment_id) AS total_appointments
FROM doctor_details d
JOIN appointment_details ad ON d.doctor_id = ad.doctor_id
GROUP BY d.doctor_name;


RESULT

#### QUERY PLAN

HashAggregate  (cost=3.34..3.54 rows=20 width=226) (actual time=0.090..0.097 rows=20 loops=1)
  Group Key: d.doctor_name
  Batches: 1  Memory Usage: 24kB
  ->  Hash Join  (cost=1.45..3.09 rows=50 width=222) (actual time=0.039..0.062 rows=50 loops=1)
        Hash Cond: (ad.doctor_id = d.doctor_id)
        ->  Seq Scan on appointment_details ad  (cost=0.00..1.50 rows=50 width=8) (actual time=0.012..0.017 rows=50 loops=1)
        ->  Hash  (cost=1.20..1.20 rows=20 width=222) (actual time=0.019..0.020 rows=20 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 10kB
              ->  Seq Scan on doctor_details d  (cost=0.00..1.20 rows=20 width=222) (actual time=0.006..0.010 rows=20 loops=1)
              
Planning Time: 0.187 ms
Execution Time: 0.140 ms

#### ANALYSIS

•  Efficient Hash Joins: The hash join between appointment_details and doctor_details is efficient, given the small number of rows being processed (50 and 20 rows, respectively).
•  Sequential Scans: Both the appointment_details and doctor_details tables are scanned sequentially. While this is efficient with small datasets, adding indexes would improve performance as the data grows.
•  HashAggregate: The aggregation (grouping by doctor_name) is performed efficiently, and the query processes the grouping without any performance bottlenecks.

Exceution time Reduced from 0.688 to 0.140 ms


### Query 5: Retrieve All Critical Severity Patients with Specific Doctor Assignments

EXPLAIN ANALYZE 
SELECT p.hosp_nis, p.key_nis, d.doctor_name, hs.aprdrg_severity
FROM patient_data p
JOIN appointment_details ad ON p.key_nis = ad.key_nis
JOIN doctor_details d ON ad.doctor_id = d.doctor_id
JOIN hospital_severity hs ON p.key_nis = hs.key_nis
WHERE hs.aprdrg_severity = 'Critical';

Result

#### QUERY PLAN

Nested Loop  (cost=2.67..23.77 rows=17 width=230) (actual time=0.086..0.137 rows=12 loops=1)
  ->  Nested Loop  (cost=2.53..20.47 rows=12 width=230) (actual time=0.080..0.112 rows=12 loops=1)
        ->  Hash Join  (cost=2.39..14.43 rows=12 width=16) (actual time=0.048..0.063 rows=12 loops=1)
              Hash Cond: (ad.key_nis = hs.key_nis)
              ->  Seq Scan on appointment_details ad  (cost=0.00..11.40 rows=140 width=8) (actual time=0.007..0.012 rows=50 loops=1)
              ->  Hash  (cost=2.24..2.24 rows=12 width=8) (actual time=0.030..0.030 rows=12 loops=1)
                    Buckets: 1024  Batches: 1  Memory Usage: 9kB
                    ->  Seq Scan on hospital_severity hs  (cost=0.00..2.24 rows=12 width=8) (actual time=0.014..0.021 rows=12 loops=1)
                          Filter: ((aprdrg_severity)::text = 'Critical'::text)
                          Rows Removed by Filter: 87
        ->  Index Scan using doctor_details_pkey on doctor_details d  (cost=0.14..0.50 rows=1 width=222) (actual time=0.003..0.003 rows=1 loops=12)
              Index Cond: (doctor_id = ad.doctor_id)
  ->  Index Scan using patient_data_pkey on patient_data p  (cost=0.14..0.27 rows=1 width=8) (actual time=0.001..0.001 rows=1 loops=12)
        Index Cond: (key_nis = ad.key_nis)
        
Planning Time: 0.542 ms
Execution Time: 0.207 ms


#### ANALYSIS

1. Nested Loop Joins: Nested loops are used to join the patient_data, appointment_details, and doctor_details tables. Nested loops work well here due to the small dataset size, as each iteration only processes a small number of rows.
2. Hash Join: The Hash Join is performed between appointment_details and hospital_severity based on key_nis. This join efficiently handles filtering and matching for the key_nis values.
3. Index Scans: Index Scans are used for both doctor_details and patient_data, utilizing the primary keys (doctor_id and key_nis) to retrieve the relevant rows quickly.



#### AFTER INDEXING 

CREATE INDEX idx_patient_data_key_nis ON patient_data (key_nis);
CREATE INDEX idx_appointment_details_key_nis ON appointment_details (key_nis);
CREATE INDEX idx_appointment_details_doctor_id ON appointment_details (doctor_id);
CREATE INDEX idx_doctor_details_doctor_id ON doctor_details (doctor_id);
CREATE INDEX idx_hospital_severity_key_nis ON hospital_severity (key_nis);
CREATE INDEX idx_hospital_severity_aprdrg_severity ON hospital_severity (aprdrg_severity);

EXPLAIN ANALYZE SELECT p.hosp_nis, p.key_nis, d.doctor_name, hs.aprdrg_severity
FROM patient_data p
JOIN appointment_details ad ON p.key_nis = ad.key_nis
JOIN doctor_details d ON ad.doctor_id = d.doctor_id
JOIN hospital_severity hs ON p.key_nis = hs.key_nis
WHERE hs.aprdrg_severity = 'Critical';

RESULT

#### QUERY PLAN

Nested Loop  (cost=4.35..8.39 rows=6 width=230) (actual time=0.082..0.111 rows=12 loops=1)
  ->  Hash Join  (cost=4.21..5.50 rows=6 width=230) (actual time=0.071..0.081 rows=12 loops=1)
        Hash Cond: (d.doctor_id = ad.doctor_id)
        ->  Seq Scan on doctor_details d  (cost=0.00..1.20 rows=20 width=222) (actual time=0.006..0.008 rows=20 loops=1)
        ->  Hash  (cost=4.13..4.13 rows=6 width=16) (actual time=0.053..0.054 rows=12 loops=1)
              Buckets: 1024  Batches: 1  Memory Usage: 9kB
              ->  Hash Join  (cost=2.39..4.13 rows=6 width=16) (actual time=0.036..0.050 rows=12 loops=1)
                    Hash Cond: (ad.key_nis = hs.key_nis)
                    ->  Seq Scan on appointment_details ad  (cost=0.00..1.50 rows=50 width=8) (actual time=0.002..0.007 rows=50 loops=1)
                    ->  Hash  (cost=2.24..2.24 rows=12 width=8) (actual time=0.027..0.027 rows=12 loops=1)
                          Buckets: 1024  Batches: 1  Memory Usage: 9kB
                          ->  Seq Scan on hospital_severity hs  (cost=0.00..2.24 rows=12 width=8) (actual time=0.014..0.020 rows=12 loops=1)
                                Filter: ((aprdrg_severity)::text = 'Critical'::text)
                                Rows Removed by Filter: 87
  ->  Index Scan using idx_patient_data_key_nis on patient_data p  (cost=0.14..0.48 rows=1 width=8) (actual time=0.002..0.002 rows=1 loops=12)
        Index Cond: (key_nis = ad.key_nis)
        
Planning Time: 0.660 ms
Execution Time: 0.155 ms


#### ANALYSIS

•  Efficient Joins: The combination of hash joins and a nested loop join works efficiently due to the small number of rows being processed. The hash joins are well-suited for the current dataset, and the index scan on patient_data speeds up the final retrieval.
•  Sequential Scans: Sequential scans are used on doctor_details, appointment_details, and hospital_severity, but the small size of the dataset makes this efficient. However, for larger datasets, indexing on the key_nis and aprdrg_severity columns would improve performance, particularly when filtering and joining large amounts of data.
•  Index Scan on patient_data: The index scan on patient_data is very efficient, as it quickly retrieves rows using the index on key_nis. This index ensures fast lookups even as the dataset grows.
Execution time reduced from 0.207 to 0.155 ms

## Conclusion: Performance Insights and Potential Improvements

Through well-integrated tables that enable thorough queries, the Hospital Management Schema efficiently manages crucial hospital activities, such as patient admissions and appointment scheduling.

#### Performance Analysis:

Useful Joins and Scans: Hash joins and sequential scans are used in a way that is appropriate for the sizes of the datasets that are currently available, guaranteeing speedy access and processing times.

Enhanced by Indexing: By cutting down on execution times and increasing operational efficiency, indexing essential columns has greatly enhanced query performance.

Possible Enhancements:
Optimize Indexing Strategy: As the database grows, make ongoing adjustments to the indexes based on usage trends to ensure peak performance.

Normalize Data: To improve data integrity and remove redundancy, further normalize tables. This could improve performance and storage.

Query and Database Optimization: Consider query rewriting, views for complex queries, and partitioning large tables to improve scalability and manageability.

These improvements can help enhance the system’s performance, scalability, and reliability, ensuring it continues to meet the demands of healthcare data management.
